<a href="https://www.kaggle.com/code/hanpat99/bitcoin-price-movement-using-coingecko-api?scriptVersionId=116601452" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Bitcoin price movement using CoinGecko API V3**

We will see how Bitcoin price moves for the last 60-days perform various analyses. We will use the **CoinGecko API** to get the price data for 60 days with 24 observation per day, 1 per hour. We will find the ***max, min, open***, and ***close price per day*** meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called PyCoinGecko. PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.

**But first, lemme install a package!**

In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance
!pip install --upgrade nbformat

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 824.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 881.2 kB/s eta 0:00:00
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.4.0
    Uninstalling nbformat-5.4.0:
      Successfully uninstalled nbformat-5.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.


Lets get the show on the road now. First, we obtain the data from **CoinGecko** using the get request, **get_coin_market_chart_by_id(id, vs_currency, days**). **id** is the name of the coin you want, **vs_currency** is the currency you want the price in, and **days** is how many days back from today you want.

Here is a description of the candle sticks.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/explanation.png" alt="Candle Stick Definition" width="800" height="600">

In [2]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days= 60)

In [3]:
type(bitcoin_data )

#Uncomment below line and execute to see the data pulled
#bitcoin_data

bitcoin_data.keys()

dict_keys(['prices', 'market_caps', 'total_volumes'])

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

In [4]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1668780086956, 16770.200488633123],
 [1668783650060, 16698.63315315737],
 [1668787309533, 16702.445985125098],
 [1668790842500, 16635.991189053635],
 [1668794490283, 16605.666730543526]]

Finally lets turn this data into a Pandas DataFrame.

In [5]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

data.head()

,TimeStamp,Price
0,1668780086956,16770.200489
1,1668783650060,16698.633153
2,1668787309533,16702.445985
3,1668790842500,16635.991189
4,1668794490283,16605.666731


Now that we have the DataFrame we will convert the **timestamp** to **datetime** and save it as a column called **date**. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [6]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [7]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})
candlestick_data

date         Price                                          
                         min           max         first          last
0   2022-11-18  16596.318344  16770.200489  16770.200489  16671.949240
1   2022-11-19  16612.085485  16749.819045  16707.022779  16749.819045
2   2022-11-20  16308.812647  16734.223063  16711.873000  16308.812647
3   2022-11-21  15665.728829  16304.076856  16304.076856  15759.214957
4   2022-11-22  15700.601322  16232.919812  15794.327605  16176.973146
..         ...           ...           ...           ...           ...
56  2023-01-13  18792.320558  19932.618003  18866.810331  19932.618003
57  2023-01-14  19941.780543  21083.034367  19941.780543  20944.019263
58  2023-01-15  20606.546176  20956.258670  20922.636839  20812.144522
59  2023-01-16  20751.537690  21339.084565  20906.040142  21196.704354
60  2023-01-17  21035.724169  21260.553260  21167.444996  21239.021440

[61 rows x 5 columns]

Finally we are now ready to use **plotly** to create our Candlestick Chart.

In [8]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

### Yay! We have built a candlestick chart for the last 60 days. You may also repeat this analysis for any number of days by updating the value passed to the parameter ****days**** in the ****get request****.

## We will be performing more interesting analyses and fun visualizations with this data in the upcoming weeks. Stay tuned!
